In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [105]:
r = requests.get('https://keithgalli.github.io/web-scraping/webpage.html')

In [106]:
soup = BeautifulSoup(r.content,'html.parser')
links = soup.find('ul',class_='socials').find_all('li') 

In [30]:
for link in links:
    print(link.find('a').get('href'))

https://www.instagram.com/keithgalli/
https://twitter.com/keithgalli
https://www.linkedin.com/in/keithgalli/
https://www.tiktok.com/@keithgalli


In [48]:
links = soup.select('ul.socials a')

In [49]:
for link in links:
    print(link.get('href'))

https://www.instagram.com/keithgalli/
https://twitter.com/keithgalli
https://www.linkedin.com/in/keithgalli/
https://www.tiktok.com/@keithgalli


In [46]:
links = soup.select('li.social a')


In [47]:
for link in links:
    print(link['href'])

https://www.instagram.com/keithgalli/
https://twitter.com/keithgalli
https://www.linkedin.com/in/keithgalli/
https://www.tiktok.com/@keithgalli


In [20]:
table = soup.find('table',class_='hockey-stats')


In [22]:
columns = table.find('thead')
columns

<thead>
<tr>
<th class="season" data-sort="">S</th>
<th class="team" data-sort="team">Team</th>
<th class="league" data-sort="league">League</th>
<th class="regular gp" data-sort="gp">GP</th>
<th class="regular g" data-sort="g">G</th>
<th class="regular a" data-sort="a">A</th>
<th class="regular tp" data-sort="tp">TP</th>
<th class="regular pim" data-sort="pim">PIM</th>
<th class="regular pm" data-sort="pm">+/-</th>
<th class="separator"> </th>
<th class="postseason">POST</th>
<th class="postseason gp" data-sort="playoffs-gp">GP</th>
<th class="postseason g" data-sort="playoffs-g">G</th>
<th class="postseason a" data-sort="playoffs-a">A</th>
<th class="postseason tp" data-sort="playoffs-tp">TP</th>
<th class="postseason pim" data-sort="playoffs-pim">PIM</th>
<th class="postseason pm" data-sort="playoffs-pm">+/-</th>
</tr>
</thead>

In [23]:
heads = [head.text for head in columns.find_all('th')]
heads

['S',
 'Team',
 'League',
 'GP',
 'G',
 'A',
 'TP',
 'PIM',
 '+/-',
 '\xa0',
 'POST',
 'GP',
 'G',
 'A',
 'TP',
 'PIM',
 '+/-']

In [24]:
# collect all the rows
contents = table.find_all('tr')

In [89]:
# remove the first row as they are heads
table_rows = contents[1:]

In [93]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n','') for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=heads)


In [94]:
df.head()

,S,Team,League,GP,G,A,TP,PIM,+/-,,POST,GP,G,A,TP,PIM,+/-
0,2014-15,MIT (Mass. Inst. of Tech.),ACHA II,17,3,9,12,20,,|,,,,,,,
1,2015-16,MIT (Mass. Inst. of Tech.),ACHA II,9,1,1,2,2,,|,,,,,,,
2,2016-17,MIT (Mass. Inst. of Tech.),ACHA II,12,5,5,10,8,0,|,,,,,,,
3,2017-18,Did not play,,,,,,,,|,,,,,,,
4,2018-19,MIT (Mass. Inst. of Tech.),ACHA III,8,5,10,15,8,,|,,,,,,,


In [4]:
# Pandas already has a built-in method to convert the table on the
# web to a dataframe:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()

,S,Team,League,GP,G,A,TP,PIM,+/-,Unnamed: 9,POST,GP.1,G.1,A.1,TP.1,PIM.1,+/-.1
0,2014-15,MIT (Mass. Inst. of Tech.),ACHA II,17.0,3.0,9.0,12.0,20.0,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-16,MIT (Mass. Inst. of Tech.),ACHA II,9.0,1.0,1.0,2.0,2.0,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-17,MIT (Mass. Inst. of Tech.),ACHA II,12.0,5.0,5.0,10.0,8.0,0.0,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-18,Did not play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-19,MIT (Mass. Inst. of Tech.),ACHA III,8.0,5.0,10.0,15.0,8.0,NaN,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
facts = soup.select('ul.fun-facts li')
facts[0].text

'Owned my dream car in high school 1'

In [106]:
facts_with_is =[fact.find(string=re.compile('is')) for fact in facts]

In [107]:
facts_with_is

[None,
 'Middle name is Ronald',
 None,
 'Dunkin Donuts coffee is better than Starbucks',
 'A favorite book series of mine is ',
 'Current video game of choice is ',
 "The band that I've seen the most times live is the "]

In [121]:
for i in facts:
    if 'is' in i.text.split():
        print(i.text)

Middle name is Ronald
Dunkin Donuts coffee is better than Starbucks
A favorite book series of mine is Ender's Game
Current video game of choice is Rocket League
The band that I've seen the most times live is the Zac Brown Band


In [15]:
add = soup.select('div.row div.column img')[0].get('src')

In [16]:
url = 'https://keithgalli.github.io/web-scraping/'
full_url = url + add
img_data = requests.get(full_url).content

In [131]:
with open('img.jpg','wb') as fh:
    fh.write(img_data)

In [86]:
links = []
link_block = soup.select('div.block a')
for link in link_block:
    links.append(link.get('href'))
        

In [71]:
link_block

[<a href="challenge/file_1.html">File 1</a>,
 <a href="challenge/file_2.html">File 2</a>,
 <a href="challenge/file_3.html">File 3</a>,
 <a href="challenge/file_4.html">File 4</a>,
 <a href="challenge/file_5.html">File 5</a>,
 <a href="challenge/file_6.html">File 6</a>,
 <a href="challenge/file_7.html">File 7</a>,
 <a href="challenge/file_8.html">File 8</a>,
 <a href="challenge/file_9.html">File 9</a>,
 <a href="challenge/file_10.html">File 10</a>]

In [88]:
for link in links:
    data = requests.get(url+link).text
    soup1 = BeautifulSoup(data,'html.parser')
    print(soup1.find('p',id='secret-word').text,end=' ')
    
    

Make sure to smash that like button and subscribe !!! 

In [99]:
link_block = soup.find_all('div',class_='block')
link_block

[<div align="left" class="block">
 <ul>
 <li><a href="challenge/file_1.html">File 1</a></li>
 <li><a href="challenge/file_2.html">File 2</a></li>
 <li><a href="challenge/file_3.html">File 3</a></li>
 <li><a href="challenge/file_4.html">File 4</a></li>
 <li><a href="challenge/file_5.html">File 5</a></li>
 </ul>
 </div>,
 <div align="center" class="block">
 <ul>
 <li><a href="challenge/file_6.html">File 6</a></li>
 <li><a href="challenge/file_7.html">File 7</a></li>
 <li><a href="challenge/file_8.html">File 8</a></li>
 <li><a href="challenge/file_9.html">File 9</a></li>
 <li><a href="challenge/file_10.html">File 10</a></li>
 </ul>
 </div>]

In [101]:
for i in link_block:
    for k in i.find_all('a'):
        print(k.get('href'))

challenge/file_1.html
challenge/file_2.html
challenge/file_3.html
challenge/file_4.html
challenge/file_5.html
challenge/file_6.html
challenge/file_7.html
challenge/file_8.html
challenge/file_9.html
challenge/file_10.html


In [103]:
link_block.find_all('a')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [108]:
link_block = soup.select('div.block ul')
link_block

[<ul>
 <li><a href="challenge/file_1.html">File 1</a></li>
 <li><a href="challenge/file_2.html">File 2</a></li>
 <li><a href="challenge/file_3.html">File 3</a></li>
 <li><a href="challenge/file_4.html">File 4</a></li>
 <li><a href="challenge/file_5.html">File 5</a></li>
 </ul>,
 <ul>
 <li><a href="challenge/file_6.html">File 6</a></li>
 <li><a href="challenge/file_7.html">File 7</a></li>
 <li><a href="challenge/file_8.html">File 8</a></li>
 <li><a href="challenge/file_9.html">File 9</a></li>
 <li><a href="challenge/file_10.html">File 10</a></li>
 </ul>]

In [111]:
for link in link_block[0]:
    print(link.get_property('href'))

AttributeError: 'NavigableString' object has no attribute 'get_property'